# Data Mining Lab : Experiment 7
### Submitted By
> NAME: Debatreya Das <br>
> Roll-No.: 12212070 <br>
> CS-A4 <br>
> Data Mining Lab

## Objective: 
### To compute maximal frequent itemset.

## Part A


Compute item set of frequent items (1-itemsets) and their support counts from a 
given transactional dataset. Sort frequent itemsets and generate them in L order 
(descending order of support counts).

#### Importing Libraries

In [36]:
## IMPORTING LIBRARIES
import pandas as pd
from collections import defaultdict

#### Loading Transaction Data

In [37]:
# Load the CSV file containing transactions
file_path = 'fp_growth_transactions.csv'  # Update with the correct path
df_transactions = pd.read_csv(file_path)
df_transactions

,TID,Items
0,T1,"bread,milk"
1,T2,"bread,diaper,beer,eggs"
2,T3,"milk,diaper,beer,cola"
3,T4,"bread,milk,diaper,beer"
4,T5,"bread,milk,cola"


#### Function to compute support of 1-itemsets `compute_1_itemsets`

In [38]:
# Function to compute support counts of 1-itemsets
def compute_1_itemsets(transactions):
    item_support_count = defaultdict(int)
    for transaction in transactions:
        items = transaction.split(',')
        for item in items:
            item_support_count[item] += 1
    return dict(item_support_count)

#### Calculating frequent 1-itemsets

In [39]:
# Extract transactions from the CSV data
transaction_list = df_transactions['Items'].tolist()

# Compute 1-itemsets with support counts
item_support_counts = compute_1_itemsets(transaction_list)

# Sorting the 1-itemsets by support counts in descending order
sorted_item_support_counts = sorted(item_support_counts.items(), key=lambda x: x[1], reverse=True)

# Display the sorted frequent 1-itemsets
sorted_item_support_counts

[('bread', 4),
 ('milk', 4),
 ('diaper', 3),
 ('beer', 3),
 ('cola', 2),
 ('eggs', 1)]

## Part B

Sort items in the transactions of the dataset in L-order (descending order of support counts).

#### Function to sort items in each transaction by L-order

In [40]:
# Helper function to sort items in each transaction by L-order (support counts)
def sort_transactions_by_l_order(transactions, support_counts):
    sorted_transactions = []
    for transaction in transactions:
        items = transaction.split(',')
        # Sort items in transaction based on the L-order (support counts)
        sorted_items = sorted(items, key=lambda item: support_counts[item], reverse=True)
        sorted_transactions.append(sorted_items)
    return sorted_transactions


#### Calculating sorted transaction by L order

In [41]:
# Sorting transactions by L-order
sorted_transactions = sort_transactions_by_l_order(transaction_list, dict(sorted_item_support_counts))

# Display the sorted transactions
sorted_transactions

[['bread', 'milk'],
 ['bread', 'diaper', 'beer', 'eggs'],
 ['milk', 'diaper', 'beer', 'cola'],
 ['bread', 'milk', 'diaper', 'beer'],
 ['bread', 'milk', 'cola']]

## Part C


Construct FP tree using the Han’s book example. Display FP tree using appropriate 
notation/representation.

#### Class: FPTreeNode

In [42]:
# FP-Tree Node structure
class FPTreeNode:
    def __init__(self, item_name, count, parent):
        self.item_name = item_name
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None  # Link to next node of the same item

    def increment(self, count):
        """Increment the count of the node."""
        self.count += count

# FP-Tree structure
class FPTree:
    def __init__(self):
        self.root = FPTreeNode(None, 1, None)  # Root node with no item
        self.header_table = {}

    def update_header_table(self, node, item):
        """Update header table to point to nodes of the same item."""
        if item in self.header_table:
            current_node = self.header_table[item]
            while current_node.link is not None:
                current_node = current_node.link
            current_node.link = node
        else:
            self.header_table[item] = node

    def insert_transaction(self, transaction):
        """Insert a sorted transaction into the FP-Tree."""
        current_node = self.root
        for item in transaction:
            if item in current_node.children:
                current_node.children[item].increment(1)
            else:
                new_node = FPTreeNode(item, 1, current_node)
                current_node.children[item] = new_node
                self.update_header_table(new_node, item)
            current_node = current_node.children[item]


#### Function to construct the FP-Tree from sorted transactions

In [43]:
# Construct the FP-Tree from sorted transactions
def construct_fp_tree(transactions):
    tree = FPTree()
    for transaction in transactions:
        tree.insert_transaction(transaction)
    return tree

#### Build the FP Growth Tree

In [44]:
# Build the FP-Tree from sorted transactions
fp_tree = construct_fp_tree(sorted_transactions)

#### Printing the tree

In [45]:
def print_fp_tree(node, indent=0):
    print('  ' * indent + f'{node.item_name}: {node.count}')
    for child in node.children.values():
        print_fp_tree(child, indent + 1)

# Print the FP-Tree structure
print_fp_tree(fp_tree.root)

None: 1
  bread: 4
    milk: 3
      diaper: 1
        beer: 1
      cola: 1
    diaper: 1
      beer: 1
        eggs: 1
  milk: 1
    diaper: 1
      beer: 1
        cola: 1


## Part D

Using FP tree, construct pattern bases and conditional FP trees.

In [46]:
# Function to extract the conditional pattern base for an item
def find_prefix_paths(base_item, header_table):
    cond_pattern_base = []
    node = header_table[base_item]
    while node is not None:
        prefix_path = []
        current_node = node
        while current_node.parent.item_name is not None:
            prefix_path.append(current_node.parent.item_name)
            current_node = current_node.parent
        if len(prefix_path) > 0:
            cond_pattern_base.append((prefix_path, node.count))
        node = node.link
    return cond_pattern_base

#### Finding Pattern Bases for all Items

In [47]:
Items = ["bread", "milk", "diaper", "beer", "eggs", "cola"]

PatternBases = {
    "bread": find_prefix_paths("bread", fp_tree.header_table),
    "milk": find_prefix_paths("milk", fp_tree.header_table),
    "diaper": find_prefix_paths("diaper", fp_tree.header_table),
    "beer": find_prefix_paths("beer", fp_tree.header_table),
    "eggs": find_prefix_paths("eggs", fp_tree.header_table),
    "cola": find_prefix_paths("cola", fp_tree.header_table)
}

for i in Items:
    print(f"PatternBases for {i}: ", PatternBases[i])

PatternBases for bread:  []
PatternBases for milk:  [(['bread'], 3)]
PatternBases for diaper:  [(['bread'], 1), (['milk'], 1), (['milk', 'bread'], 1)]
PatternBases for beer:  [(['diaper', 'bread'], 1), (['diaper', 'milk'], 1), (['diaper', 'milk', 'bread'], 1)]
PatternBases for eggs:  [(['beer', 'diaper', 'bread'], 1)]
PatternBases for cola:  [(['beer', 'diaper', 'milk'], 1), (['milk', 'bread'], 1)]


## Part E

Generate frequent patterns.

#### Helper Functions

In [48]:
from collections import defaultdict

# Function to filter items by support threshold
def filter_items_by_support(transactions, min_support):
    item_count = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_count[item] += 1

    # Filter out items that don't meet the minimum support
    filtered_items = {item for item, count in item_count.items() if count >= min_support}
    return filtered_items, item_count

#### Function to generate the conditional fp-trees

In [49]:
# Function to create a conditional FP-Tree
def construct_conditional_fp_tree(base_item, cond_pattern_base, min_support):
    tree = FPTree()
    for prefix_path, count in cond_pattern_base:
        # Filter out items that don't meet min support in prefix path
        filtered_prefix_path = [item for item in prefix_path if item in tree.header_table]
        filtered_prefix_path.sort(key=lambda item: tree.header_table[item].count, reverse=True)
        for _ in range(count):
            tree.insert_transaction(filtered_prefix_path)
    return tree

# Function to mine frequent patterns using conditional FP-trees
def mine_fp_tree(tree, min_support, prefix, frequent_patterns):
    # Sort items in the header table by frequency
    sorted_items = sorted(tree.header_table.items(), key=lambda x: x[1].count)

    for base_item, node in sorted_items:
        new_prefix = prefix.copy()
        
        # Ensure no duplicate items in the new prefix
        if base_item not in new_prefix:
            new_prefix.append(base_item)

            # Calculate total support for the current pattern
            total_support = 0
            current_node = node
            while current_node is not None:
                total_support += current_node.count
                current_node = current_node.link

            # Add the frequent pattern (prefix + item) to the result
            if total_support >= min_support:
                frequent_patterns.append((new_prefix, total_support))

            # Find conditional pattern base
            cond_pattern_base = find_prefix_paths(base_item, tree.header_table)

            # Construct conditional FP-Tree for current item
            cond_tree = FPTree()
            for path, count in cond_pattern_base:
                cond_tree.insert_transaction(path * count)  # Insert with counts

            # Recursively mine the conditional FP-tree
            if cond_tree.root.children:
                mine_fp_tree(cond_tree, min_support, new_prefix, frequent_patterns)


#### Generate Frequent Patterns

In [50]:
# Main driver function
def generate_frequent_patterns(transactions, min_support):
    # Step 1: Filter items by min_support
    filtered_items, item_count = filter_items_by_support(transactions, min_support)

    # Step 2: Filtered transactions
    filtered_transactions = [[item for item in transaction if item in filtered_items]
                             for transaction in transactions]

    # Step 3: Construct the initial FP-Tree
    tree = construct_fp_tree(filtered_transactions)

    # Step 4: Recursively mine the FP-Tree
    frequent_patterns = []
    mine_fp_tree(tree, min_support, [], frequent_patterns)

    return frequent_patterns

In [51]:
# DRIVER CODE
transactions = [items.split(',') for items in df_transactions['Items']]
min_support = 2
frequent_patterns = generate_frequent_patterns(transactions, min_support)


# Display all frequent patterns
for pattern, count in frequent_patterns:
    print(f"Pattern: {pattern}, Support: {count}")

Pattern: ['diaper'], Support: 3
Pattern: ['diaper', 'bread'], Support: 2
Pattern: ['diaper', 'milk'], Support: 2
Pattern: ['beer'], Support: 3
Pattern: ['beer', 'bread'], Support: 2
Pattern: ['beer', 'bread', 'diaper'], Support: 2
Pattern: ['beer', 'milk'], Support: 2
Pattern: ['beer', 'milk', 'diaper'], Support: 2
Pattern: ['beer', 'diaper'], Support: 3
Pattern: ['cola'], Support: 2
Pattern: ['cola', 'milk'], Support: 2
Pattern: ['milk'], Support: 4
Pattern: ['milk', 'bread'], Support: 3
Pattern: ['bread'], Support: 4
